# FinBERT Profiling: Medium FinBert + Knowledge Distilation

This notebook is intentionally **thin**: it reuses the profiling utilities in `pipelines/finBERT/finbert/` (especially `finbert/finbert_profile.py` and `finbert/profile_utils.py`) instead of copying large code blocks.

The purpose of this notebook is to profile finetuning and distilling a 'medium-bert' model on financial sentiment analysis


In [3]:
from __future__ import annotations

from pathlib import Path
import shutil
import os
import logging
import sys
sys.path.append('..')

from textblob import TextBlob
from pprint import pprint
from sklearn.metrics import classification_report

from transformers import AutoModelForSequenceClassification

from finbert.finbert import *
from finbert.finbert_profile import *
from finbert.profile_utils import get_model_size_mb, print_device_info, setup_nltk_data, timed_eval
import finbert.utils as tools

from finbert.finbert_kd_trainer import KDFinBert



%load_ext autoreload
%autoreload 2

project_dir = Path.cwd().parent
pd.set_option('max_colwidth', None)

import wandb

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
wandb.init(
    entity="si2449-columbia-university",
    project="finbert-experiments",
   name="finetuning/distilling-medium_bert",
    group="knowledge-distillation",
)

In [5]:
cl_path = project_dir/'models'/'student'
cl_data_path = project_dir/'data'/'sentiment_data'

In [6]:
try:
    shutil.rmtree(cl_path) 
except:
    pass

bertmodel = AutoModelForSequenceClassification.from_pretrained("prajjwal1/bert-medium", cache_dir=None, num_labels=3)


config = Config(   data_dir=cl_data_path,
                   bert_model=bertmodel,
                   num_train_epochs=4,
                   model_dir=cl_path,
                   max_seq_length = 48,
                   train_batch_size = 32,
                   learning_rate = 2e-5,
                   output_mode='classification',
                   warm_up_proportion=0.2,
                   local_rank=-1,
                   discriminate=True,
                   gradual_unfreeze=True,
                   encoder_no = 8)

config.profile_train_steps = 20
config.num_hidden_layers = 8

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-medium and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
finbert = ProfiledFinBert(config)
finbert.config.base_model = 'prajjwal1/bert-medium'
finbert.config.discriminate=True
finbert.config.gradual_unfreeze=True

In [8]:
finbert.prepare_model(label_list=['positive','negative','neutral'])

12/16/2025 18:29:35 - INFO - finbert.finbert -   device: cuda n_gpu: 1, distributed training: False, 16-bits training: False


In [9]:
train_data = finbert.get_data('train')

In [10]:
model = finbert.create_the_model()

In [11]:
start = time.perf_counter()
trained_model = finbert.train(train_examples = train_data, model = model)
train_wall_s = time.perf_counter() - start

12/16/2025 18:29:38 - INFO - finbert.utils -   *** Example ***
12/16/2025 18:29:38 - INFO - finbert.utils -   guid: train-1
12/16/2025 18:29:38 - INFO - finbert.utils -   tokens: [CLS] after the reporting period , bio ##tie north american licensing partner so ##max ##on pharmaceuticals announced positive results with na ##lm ##efe ##ne in a pilot phase 2 clinical trial for smoking ce ##ssa ##tion [SEP]
12/16/2025 18:29:38 - INFO - finbert.utils -   input_ids: 101 2044 1996 7316 2558 1010 16012 9515 2167 2137 13202 4256 2061 17848 2239 24797 2623 3893 3463 2007 6583 13728 27235 2638 1999 1037 4405 4403 1016 6612 3979 2005 9422 8292 11488 3508 102 0 0 0 0 0 0 0 0 0 0 0
12/16/2025 18:29:38 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0
12/16/2025 18:29:38 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
12/16/

12/16/2025 18:29:38 - INFO - finbert.finbert -   ***** Loading data *****
12/16/2025 18:29:38 - INFO - finbert.finbert -     Num examples = 3488
12/16/2025 18:29:38 - INFO - finbert.finbert -     Batch size = 32
12/16/2025 18:29:38 - INFO - finbert.finbert -     Num steps = 48



Starting Profiled Training
Device: cuda
Profiling activities: [<ProfilerActivity.CPU: 0>, <ProfilerActivity.CUDA: 2>]



Epoch:   0%|          | 0/4 [00:00<?, ?it/s]



Profiling complete for first epoch (20 steps)
Continuing full training without profiling...


PROFILING RESULTS - Training


By CPU Time:
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg       CPU Mem  Self CPU Mem      CUDA Mem  Self CUDA Mem    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                           forward_pass         7.71%      71.651ms        41.47%

...

Iteration: 100%|██████████| 109/109 [00:08<00:00, 12.49it/s]
12/16/2025 18:30:08 - INFO - finbert.utils -   *** Example ***
12/16/2025 18:30:08 - INFO - finbert.utils -   guid: validation-1
12/16/2025 18:30:08 - INFO - finbert.utils -   tokens: [CLS] our in - depth expertise extends to the fields of energy , industry , urban & mobility and water & environment [SEP]
12/16/2025 18:30:08 - INFO - finbert.utils -   input_ids: 101 2256 1999 1011 5995 11532 8908 2000 1996 4249 1997 2943 1010 3068 1010 3923 1004 12969 1998 2300 1004 4044 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
12/16/2025 18:30:08 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
12/16/2025 18:30:08 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
12/16/2025 18:30:08 - INFO - finbert.utils -   label: neutral (id = 2)
12/16/2025 18:30:08 

Validation losses: [0.9901655729000385, 0.7837056334202106, 0.6307268303174239]


Iteration: 100%|██████████| 109/109 [00:09<00:00, 12.08it/s]
12/16/2025 18:30:17 - INFO - finbert.utils -   *** Example ***
12/16/2025 18:30:17 - INFO - finbert.utils -   guid: validation-1
12/16/2025 18:30:17 - INFO - finbert.utils -   tokens: [CLS] our in - depth expertise extends to the fields of energy , industry , urban & mobility and water & environment [SEP]
12/16/2025 18:30:17 - INFO - finbert.utils -   input_ids: 101 2256 1999 1011 5995 11532 8908 2000 1996 4249 1997 2943 1010 3068 1010 3923 1004 12969 1998 2300 1004 4044 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
12/16/2025 18:30:17 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
12/16/2025 18:30:17 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
12/16/2025 18:30:17 - INFO - finbert.utils -   label: neutral (id = 2)
12/16/2025 18:30:18 

Validation losses: [0.9901655729000385, 0.7837056334202106, 0.6307268303174239, 0.5875818591851455]


Epoch: 100%|██████████| 4/4 [00:31<00:00,  7.89s/it]


In [12]:
test_data = finbert.get_data("test")

results = finbert.evaluate(examples=test_data, model=trained_model)

eval_df, eval_timing = timed_eval(
    finbert=finbert, model=trained_model, examples=test_data, use_amp=False
)

12/16/2025 18:30:19 - INFO - finbert.utils -   *** Example ***
12/16/2025 18:30:19 - INFO - finbert.utils -   guid: test-1
12/16/2025 18:30:19 - INFO - finbert.utils -   tokens: [CLS] the bristol port company has sealed a one million pound contract with cooper specialised handling to supply it with four 45 - ton ##ne , custom ##ised reach stack ##ers from ko ##ne ##cr ##ane ##s [SEP]
12/16/2025 18:30:19 - INFO - finbert.utils -   input_ids: 101 1996 7067 3417 2194 2038 10203 1037 2028 2454 9044 3206 2007 6201 17009 8304 2000 4425 2009 2007 2176 3429 1011 10228 2638 1010 7661 5084 3362 9991 2545 2013 12849 2638 26775 7231 2015 102 0 0 0 0 0 0 0 0 0 0
12/16/2025 18:30:19 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0
12/16/2025 18:30:19 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
12/16/2025 18:30:19 - IN

Testing: 100%|██████████| 31/31 [00:00<00:00, 38.21it/s]
12/16/2025 18:30:20 - INFO - finbert.utils -   *** Example ***
12/16/2025 18:30:20 - INFO - finbert.utils -   guid: test-1
12/16/2025 18:30:20 - INFO - finbert.utils -   tokens: [CLS] the bristol port company has sealed a one million pound contract with cooper specialised handling to supply it with four 45 - ton ##ne , custom ##ised reach stack ##ers from ko ##ne ##cr ##ane ##s [SEP]
12/16/2025 18:30:20 - INFO - finbert.utils -   input_ids: 101 1996 7067 3417 2194 2038 10203 1037 2028 2454 9044 3206 2007 6201 17009 8304 2000 4425 2009 2007 2176 3429 1011 10228 2638 1010 7661 5084 3362 9991 2545 2013 12849 2638 26775 7231 2015 102 0 0 0 0 0 0 0 0 0 0
12/16/2025 18:30:20 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0
12/16/2025 18:30:20 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [13]:
def report(df, cols=['label','prediction','logits']):
    #print('Validation loss:{0:.2f}'.format(metrics['best_validation_loss']))
    cs = CrossEntropyLoss(weight=finbert.class_weights)
    loss = cs(torch.tensor(list(df[cols[2]])),torch.tensor(list(df[cols[0]])))
    print("Evaluation Loss:{0:.2f}".format(loss))
    print("Evaluation Accuracy:{0:.2f}".format((df[cols[0]] == df[cols[1]]).sum() / df.shape[0]) )
    print("\nClassification Report:")
    return_val = classification_report(df[cols[0]], df[cols[1]], output_dict=True)
    
    
    new_report = {}
    
    for key in return_val.keys():
        new_report["Finetuning " + key] = return_val[key]
    
    new_report["Finetuning Evalaution Loss"] = loss
    new_report["Fintuning Evaluation Accuracy"] = (df[cols[0]] == df[cols[1]]).sum() / df.shape[0]
    
    
    print(new_report)
        
    return new_report

In [14]:
results['prediction'] = results.predictions.apply(lambda x: np.argmax(x,axis=0))

In [15]:
wandb_report = report(results,cols=['labels','prediction','predictions'])


Evaluation Loss:0.63
Evaluation Accuracy:0.75

Classification Report:
{'Finetuning 0': {'precision': 0.6498194945848376, 'recall': 0.6741573033707865, 'f1-score': 0.6617647058823529, 'support': 267.0}, 'Finetuning 1': {'precision': 0.5661375661375662, 'recall': 0.8359375, 'f1-score': 0.6750788643533123, 'support': 128.0}, 'Finetuning 2': {'precision': 0.873015873015873, 'recall': 0.7652173913043478, 'f1-score': 0.8155699721964782, 'support': 575.0}, 'Finetuning accuracy': 0.7494845360824742, 'Finetuning macro avg': {'precision': 0.6963243112460923, 'recall': 0.7584373982250447, 'f1-score': 0.7174711808107146, 'support': 970.0}, 'Finetuning weighted avg': {'precision': 0.7710840623751414, 'recall': 0.7494845360824742, 'f1-score': 0.7546948506399869, 'support': 970.0}, 'Finetuning Evalaution Loss': tensor(0.6303), 'Fintuning Evaluation Accuracy': np.float64(0.7494845360824742)}


/var/tmp/ipykernel_54311/142971867.py:4: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:253.)
  loss = cs(torch.tensor(list(df[cols[2]])),torch.tensor(list(df[cols[0]])))


In [17]:
teacher_path = project_dir/'models'/'teacher'
teacher = AutoModelForSequenceClassification.from_pretrained(
    teacher_path, num_labels=3, cache_dir=None
)
student = trained_model

new_path = project_dir/'models'/'distilled_student'

In [18]:
config = Config(   data_dir=cl_data_path,
                   bert_model=None,
                   num_train_epochs=2,
                   model_dir=new_path,
                   max_seq_length = 48,
                   train_batch_size = 32,
                   learning_rate = 5e-6,
                   output_mode='classification',
                   warm_up_proportion=0.1,
                   local_rank=-1,
                   discriminate=True,
                   gradual_unfreeze=True,
                   encoder_no = 4,
                   base_model='prajjwal1/bert-medium')


In [19]:
kd = KDFinBert(teacher=teacher, student=student, config=config)

In [20]:
kd.prepare_model(label_list=["positive", "negative", "neutral"])

In [21]:
train_data = kd.get_data('train')
model = kd.create_the_model()

In [22]:
start = time.perf_counter()
trained_model = kd.train(train_examples = train_data, teacher=teacher, student=student)
train_wall_s = time.perf_counter() - start

12/16/2025 18:33:25 - INFO - finbert.utils -   *** Example ***
12/16/2025 18:33:25 - INFO - finbert.utils -   guid: train-1
12/16/2025 18:33:25 - INFO - finbert.utils -   tokens: [CLS] after the reporting period , bio ##tie north american licensing partner so ##max ##on pharmaceuticals announced positive results with na ##lm ##efe ##ne in a pilot phase 2 clinical trial for smoking ce ##ssa ##tion [SEP]
12/16/2025 18:33:25 - INFO - finbert.utils -   input_ids: 101 2044 1996 7316 2558 1010 16012 9515 2167 2137 13202 4256 2061 17848 2239 24797 2623 3893 3463 2007 6583 13728 27235 2638 1999 1037 4405 4403 1016 6612 3979 2005 9422 8292 11488 3508 102 0 0 0 0 0 0 0 0 0 0 0
12/16/2025 18:33:25 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0
12/16/2025 18:33:25 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
12/16/

12/16/2025 18:33:26 - INFO - finbert.finbert_kd_trainer -   ***** Loading data *****
12/16/2025 18:33:26 - INFO - finbert.finbert_kd_trainer -     Num examples = 3488
12/16/2025 18:33:26 - INFO - finbert.finbert_kd_trainer -     Batch size = 32
12/16/2025 18:33:26 - INFO - finbert.finbert_kd_trainer -     Num steps = 24
/home/srm2245/hpml-project/pipelines/finBERT/notebooks/../finbert/finbert_kd_trainer.py:314: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=self.config.use_amp)
Epoch:   0%|          | 0/2 [00:00<?, ?it/s]/home/srm2245/hpml-project/pipelines/finBERT/notebooks/../finbert/finbert_kd_trainer.py:356: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=self.config.use_amp):
Epoch:   0%|          | 0/2 [00:00<?, ?it/s]/home/srm2245/hpml-project/

Validation losses: [0.5462722892944629]


Iteration: 100%|██████████| 109/109 [00:18<00:00,  5.81it/s]
12/16/2025 18:34:27 - INFO - finbert.utils -   *** Example ***
12/16/2025 18:34:27 - INFO - finbert.utils -   guid: validation-1
12/16/2025 18:34:27 - INFO - finbert.utils -   tokens: [CLS] our in - depth expertise extends to the fields of energy , industry , urban & mobility and water & environment [SEP]
12/16/2025 18:34:27 - INFO - finbert.utils -   input_ids: 101 2256 1999 1011 5995 11532 8908 2000 1996 4249 1997 2943 1010 3068 1010 3923 1004 12969 1998 2300 1004 4044 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
12/16/2025 18:34:27 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
12/16/2025 18:34:27 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
12/16/2025 18:34:27 - INFO - finbert.utils -   label: neutral (id = 2)
12/16/2025 18:34:27 

Validation losses: [0.5462722892944629, 0.5244703545020177]


Epoch: 100%|██████████| 2/2 [00:38<00:00, 19.40s/it]


In [23]:
test_data = kd.get_data("test")

results = kd.evaluate(examples=test_data, model=trained_model)



12/16/2025 18:34:28 - INFO - finbert.utils -   *** Example ***
12/16/2025 18:34:28 - INFO - finbert.utils -   guid: test-1
12/16/2025 18:34:28 - INFO - finbert.utils -   tokens: [CLS] the bristol port company has sealed a one million pound contract with cooper specialised handling to supply it with four 45 - ton ##ne , custom ##ised reach stack ##ers from ko ##ne ##cr ##ane ##s [SEP]
12/16/2025 18:34:28 - INFO - finbert.utils -   input_ids: 101 1996 7067 3417 2194 2038 10203 1037 2028 2454 9044 3206 2007 6201 17009 8304 2000 4425 2009 2007 2176 3429 1011 10228 2638 1010 7661 5084 3362 9991 2545 2013 12849 2638 26775 7231 2015 102 0 0 0 0 0 0 0 0 0 0
12/16/2025 18:34:28 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0
12/16/2025 18:34:28 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
12/16/2025 18:34:28 - IN

12/16/2025 18:34:28 - INFO - finbert.finbert_kd_trainer -     Num examples = 970
12/16/2025 18:34:28 - INFO - finbert.finbert_kd_trainer -     Batch size = 32
12/16/2025 18:34:28 - INFO - finbert.finbert_kd_trainer -     Num steps = 60
12/16/2025 18:34:28 - INFO - finbert.finbert_kd_trainer -   ***** Running evaluation ***** 
12/16/2025 18:34:28 - INFO - finbert.finbert_kd_trainer -     Num examples = 970
12/16/2025 18:34:28 - INFO - finbert.finbert_kd_trainer -     Batch size = 32
Testing: 100%|██████████| 31/31 [00:00<00:00, 38.12it/s]


In [24]:
results['prediction'] = results.predictions.apply(lambda x: np.argmax(x,axis=0))

In [25]:
def kd_report(df, cols=['label','prediction','logits']):
    #print('Validation loss:{0:.2f}'.format(metrics['best_validation_loss']))
    cs = CrossEntropyLoss(weight=finbert.class_weights)
    loss = cs(torch.tensor(list(df[cols[2]])),torch.tensor(list(df[cols[0]])))
    print("Loss:{0:.2f}".format(loss))
    print("Accuracy:{0:.2f}".format((df[cols[0]] == df[cols[1]]).sum() / df.shape[0]) )
    print("\nClassification Report:")
    return_val = classification_report(df[cols[0]], df[cols[1]], output_dict=True)
    
    return_val["Evaluation Loss"] = loss
    return_val["Evaluation Accuracy"] = (df[cols[0]] == df[cols[1]]).sum() / df.shape[0]
    return return_val

In [26]:
wandb_kd_report = kd_report(results,cols=['labels','prediction','predictions'])

summary = {
        "device": str(finbert.device),
        "model_dir": str(cl_path),
        "train_wall_s": float(train_wall_s),
        "train_examples": int(len(train_data)),
        "train_examples_per_s": float((len(train_data) * finbert.config.num_train_epochs) / train_wall_s)
        if train_wall_s > 0
        else float("inf"),
        "model_size_mb": float(get_model_size_mb(trained_model)),
        "profile_train_steps": finbert.config.profile_train_steps,
        **(finbert.profile_results.get("training_summary", {}) or {}),
        **eval_timing,
        **wandb_report,
        **wandb_kd_report,
    }

wandb.log(summary)
wandb.finish()

Loss:0.56
Accuracy:0.77

Classification Report:


Evaluation Accuracy,▁
Evaluation Loss,▁
Finetuning Evalaution Loss,▁
Finetuning accuracy,▁
Fintuning Evaluation Accuracy,▁
accuracy,▁
epoch,▁▁▁▁▁▁▁▁▁▁▃▃▃▃▃▃▃▃▃▃▆▆▆▆▆▆▆▆▆▆██████████
eval_num_samples,▁
eval_samples_per_s,▁
eval_wall_s,▁
+14,...
